# Prototype 1
**Stack:**
```
OCR: Gemini 2.5 flash
Embeddings: gemini-embedding-exp-03-07
LLM: Gemini 2.5 flash

```
It requires credentials, you will need to download googles authentication CLI. Check their [official documentation](https://cloud.google.com/vision/docs/authentication)

> Installation

In [72]:
%pip install --upgrade google-genai
%pip install python-dotenv
%pip install requests
%pip install faiss-cpu #or faiss-gpu 


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [73]:
#Variables
from dotenv import load_dotenv
load_dotenv()
import os
OCR_API_KEY=os.environ.get("GCV_API_KEY")
EMBEDDINGS_API_KEY=os.environ.get("GEMINI_API_KEY")
MILO_API_KEY=os.environ.get("MILO_API_KEY")
doc_path="Lab_result_pseudoAno.pdf"
questionnaire_id = 1701  # Update this to the correct questionnaire ID

> OCR

In [74]:
import os
from google import genai
from google.genai.types import (
    GenerateContentConfig,
    HarmBlockThreshold,
    HarmCategory,
    HttpOptions,
    Part,
    SafetySetting,
)


client = genai.Client(http_options=HttpOptions(api_version="v1"), api_key=OCR_API_KEY)


# Read the PDF file as bytes
with open(doc_path, "rb") as pdf_file:
    pdf_bytes = pdf_file.read()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[
        Part.from_bytes(
            data=pdf_bytes,
            mime_type="application/pdf",
        ),
        """Extract and output the positions of objects and the text in them for each page. Return the output in JSON format without markdown fomatting. Like this:
            [
                {
        "page": 1,
        "objects": [
        {
            "box_2d": [75, 48, 89, 401],
            "text": "  "
        }
        ]
            }
        ]
        
        """,
    ],
)
print(response.text)


[
  {
    "page": 1,
    "objects": [
      {
        "box_2d": [75, 48, 89, 401],
        "text": "  "
      },
      {
        "box_2d": [75, 442, 90, 715],
        "text": " - Près d'Arènes"
      },
      {
        "box_2d": [90, 72, 100, 396],
        "text": "N° FINESS (facturation): 34 3 73132 9"
      },
      {
        "box_2d": [101, 72, 114, 476],
        "text": "730  des Pres d'Arenes - "
      },
      {
        "box_2d": [102, 482, 115, 608],
        "text": "-  ☎"
      },
      {
        "box_2d": [102, 618, 114, 927],
        "text": ":64 8255 Is.mtppresdarenes@.fr"
      },
      {
        "box_2d": [133, 150, 148, 313],
        "text": "10/04/25-DG-0042"
      },
      {
        "box_2d": [148, 48, 158, 271],
        "text": "Identifiant Labo du Patient n° B9021503046"
      },
      {
        "box_2d": [165, 48, 177, 114],
        "text": "Prescripteur"
      },
      {
        "box_2d": [176, 48, 191, 221],
        "text": "DR    "
      },
      {
        "box_2d

> Prepare the output for embeddings. 

In [75]:
import json
response_json=json.loads(response.text)
all_text = " ".join(
    obj.get("text", obj.get("text_content", "")) 
    for page in response_json 
    for obj in page["objects"]
)
all_words=all_text.split()
chunk_size=50
split_size=len(all_words) //chunk_size
embed_input=[]
for i in range(split_size):
    embed_input.append(" ".join(all_words[i*chunk_size:(i+1)*chunk_size]))
embed_input.append(" ".join(all_words[split_size*chunk_size:]))
print(embed_input)


['    - Près d\'Arènes N° FINESS (facturation): 34 3 73132 9 730  des Pres d\'Arenes -  -  ☎ :64 8255 Is.mtppresdarenes@.fr 10/04/25-DG-0042 Identifiant Labo du Patient n° B9021503046 Prescripteur DR     CABINET MEDICAL " "   Patient Nom de naissance :', '(F) Date de naissance: 25/11/1980 (45 Ans) Liste prénom(s) : 10/04/25-2-0622 Destinataire(s) du compte-rendu : INS qualifié: 293114410981345 NIR DR    , LABO 364 Prélèvements le 10/04/25 à 09H23, Edition complète le vendredi 11 avril 2025 à 18:51 (Préleveur interne au laboratoire) A partir du 10/03/25, consultation de votre compte-rendu', 'de résultats en un clic: Rapidité - Simplicité - Sécurité Pour toute demande, rapprochez vous de votre laboratoire de proximité. Information Afin de maintenir la qualité de votre prise en charge tout en nous adaptant aux nouveaux défis imposés à la Biologie Médicale, nous vous informons que certains laboratoires ', "  modifient leurs horaires d'ouverture. Retrouvez tous les laboratoires ouverts l'ap

> Getting the survey

In [ ]:
import requests

def get_questionnaire_data(questionnaire_id):
 
    # Replace with your API key
    api_key = MILO_API_KEY
    
    # Replace with the URL you want to access
    url = f"https://dev.milo-dct-backend.com:8443/api/v1/questionnaires/{questionnaire_id}/questions-ai"
 
    # Headers to include the API key
    headers = {
        "Authorization": f"Bearer {api_key}"
    }
 
    # Make the GET request
    
    response = requests.get(url, headers=headers)
 
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()  # or response.text
    else:
        print(f"Request failed with status code : {response.status_code}")
        return response.text
    
data = (get_questionnaire_data(questionnaire_id))

# Extract question headers
question_titles = [question["groupTitle"] for question in data]


[{'id': 11835, 'groupTitle': 'Blood sugar', 'groupType': 'QUESTION_GROUP', 'subQuestions': [{'id': 11837, 'questionTitle': 'result', 'questionType': 'DECIMAL'}, {'id': 11838, 'questionTitle': 'noraml values', 'questionType': 'RANGE'}, {'id': 11839, 'questionTitle': 'units', 'questionType': 'String'}]}, {'id': 11844, 'groupTitle': 'Serum Insulin', 'groupType': 'QUESTION_GROUP', 'subQuestions': [{'id': 11846, 'questionTitle': 'noraml values', 'questionType': 'RANGE'}, {'id': 11847, 'questionTitle': 'units', 'questionType': 'String'}, {'id': 11845, 'questionTitle': 'result', 'questionType': 'DECIMAL'}]}, {'id': 11840, 'groupTitle': 'HbA1c', 'groupType': 'QUESTION_GROUP', 'subQuestions': [{'id': 11841, 'questionTitle': 'result', 'questionType': 'DECIMAL'}, {'id': 11842, 'questionTitle': 'noraml values', 'questionType': 'RANGE'}, {'id': 11843, 'questionTitle': 'units', 'questionType': 'String'}]}]
['Blood sugar', 'Serum Insulin', 'HbA1c']


> Embedding

In [77]:
from google import genai
from google.genai import types
client = genai.Client(api_key=EMBEDDINGS_API_KEY)
embedded_text = client.models.embed_content(
        model="gemini-embedding-exp-03-07",
        contents=embed_input,
        config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY")
)
embedded_questions = client.models.embed_content(
        model="gemini-embedding-exp-03-07",
        contents=question_titles,
        config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY")
)

print(embedded_text.embeddings)

[ContentEmbedding(
  values=[
    -0.009169304,
    0.003619151,
    0.014024457,
    -0.06750568,
    -0.022761706,
    <... 3067 more items ...>,
  ]
), ContentEmbedding(
  values=[
    -0.014096853,
    0.003941482,
    0.0038931973,
    -0.04887911,
    -0.007232855,
    <... 3067 more items ...>,
  ]
), ContentEmbedding(
  values=[
    -0.007585881,
    0.00095533615,
    -0.0020752668,
    -0.04665719,
    -0.012652837,
    <... 3067 more items ...>,
  ]
), ContentEmbedding(
  values=[
    -0.003974031,
    -0.017716473,
    -0.00054677884,
    -0.055852316,
    -0.015811514,
    <... 3067 more items ...>,
  ]
), ContentEmbedding(
  values=[
    0.007032973,
    0.0009580488,
    0.012632609,
    -0.07071237,
    -0.011911012,
    <... 3067 more items ...>,
  ]
), ContentEmbedding(
  values=[
    0.03465852,
    -0.00076482457,
    -0.002885865,
    -0.054699637,
    -0.008627815,
    <... 3067 more items ...>,
  ]
), ContentEmbedding(
  values=[
    -0.008622181,
    0.00797183,

> Performing a similarity search with the faiss library.

In [78]:
import numpy as np
import faiss

# Convert Gemini embeddings to numpy arrays
answer_embeddings = np.array([e.values for e in embedded_text.embeddings], dtype=np.float32)
question_embeddings = np.array([e.values for e in embedded_questions.embeddings], dtype=np.float32)

dimension = answer_embeddings.shape[1]

# Normalize for cosine similarity
faiss.normalize_L2(answer_embeddings)
faiss.normalize_L2(question_embeddings)

# Create the FAISS index for cosine similarity
index = faiss.IndexFlatIP(dimension)
index.add(answer_embeddings)

# Search for the nearest neighbors
k = 3
D, I = index.search(question_embeddings, k)


> Prepare the text and parse it for the LLM

In [2]:
subQuestions = [question["subQuestions"] for question in data]

for i in range(len(subQuestions)):
    # Combine main and subquestions into a flat list
    LLMquestions = [question_titles[i]] + [q["questionTitle"] for q in subQuestions[i]]
    questions_str = "\n".join(f"- {q}" for q in LLMquestions)

    # Build context string
    LLMcontext = []
    for j in range(k):
        idx = I[i][j]
        if idx < len(embed_input):
            LLMcontext.append(embed_input[idx])
    context_str = "\n".join(LLMcontext)

    # Generate response
    response = client.models.generate_content(
        model="gemini-2.5-flash",
            contents=[
            f"""
        Answer the following questions using only the information from the context.
        Return your answers as a single JSON object without markdown quotes, with each question as a key and the answer as the value.
        If an answer is not found in the context, use "Not found in context." as the value.

        Questions:
        {questions_str}

        Context:
        {context_str}
        """
            ],
        )
    print(response.text)



NameError: name 'data' is not defined